# **Modelling most important features**

## Objectives

1. Create a model which determins house prices based on the most important features identified in the PriceCorrelationStudy notebook.
2. This can be based on the code generated in the ModellingAndEvaluation notebook.
3. Answers business criteria 2. Predict prices for client house data.
4. We have agreed an R2 score of at least 0.75 on the train set as well as on the test set.
5. Can predict house prices based on input data from streamlit dashboard. 

## Inputs

1. House_prices_records_clean.csv
2. Inherited_houses_clean.csv
3. Findings of feature engineering notebook
4. Most important features identified in the PriceCorrelationStudy notebook.

## Outputs

1. Data sets for train, validate and test sets.
2. Feature engineering pipeline.
3. Trained Model.
4. Predictions for client house prices and supporting data.

Please note that all code in this notebook is taken from The Code institute Data Analysis & Machine Learning Toolkit [Data Analysis & Machine Learning Toolkit](https://learn.codeinstitute.net/courses/course-v1:code_institute+CI_DA_ML+2021_Q4/courseware/1f851533cd6a4dcd8a280fd9f37ef4e2/81c19e89e4e94690bd58f738cb7eae91/) lesson.

## Comments

1. This process is necessary as we need a model which can predict house prices based off live data input into the streamlit dashboard. The model needs all features to be input to predict a house price. 
2. The present model needs 16 features to be provided. It will be a better user experience if we can create a model which needs fewer inputs.
3. We can use the most important features identified in the PriceCorrelationStudy notebook to create a model as accurate as one which uses more features.

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing'

# Load data

Section 1 content

In [4]:
import numpy as np
import pandas as pd

house_prices_clean_df = pd.read_csv(f"outputs/datasets/clean_data/House_prices_records_clean.csv")
house_prices_clean_df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GarageYrBlt,...,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,548,RFn,2003.0,...,8450,65.0,196.0,61,5,7,856,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,460,RFn,1976.0,...,9600,80.0,0.0,0,8,6,1262,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,608,RFn,2001.0,...,11250,68.0,162.0,42,5,7,920,2001,2002,223500
3,961,0.0,3.0,No,216,ALQ,540,642,Unf,1998.0,...,9550,60.0,0.0,35,5,7,756,1915,1970,140000
4,1145,0.0,4.0,Av,655,GLQ,490,836,RFn,2000.0,...,14260,84.0,350.0,84,5,8,1145,2000,2000,250000


---

# Split data

We need to split the data into train, validation and test sets. We keep the same 7:1:2 ratio.

First we split the data into the train and test sets

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train, y_test = train_test_split(
                                    house_prices_clean_df.drop(['SalePrice'],axis=1),
                                    house_prices_clean_df['SalePrice'],
                                    test_size=0.2,
                                    random_state=0
                                    )

print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:",  X_test.shape, y_test.shape)

* Train set: (1168, 21) (1168,) 
* Test set: (292, 21) (292,)


We then split the train set further into train and validation sets.

In [6]:
X_train, X_val,y_train, y_val = train_test_split(
                                    X_train,
                                    y_train,
                                    test_size=0.2,
                                    random_state=0
                                    )

print("* Train set:", X_train.shape, y_train.shape)
print("* Validation set:",  X_val.shape, y_val.shape)
print("* Test set:",   X_test.shape, y_test.shape)

* Train set: (934, 21) (934,)
* Validation set: (234, 21) (234,)
* Test set: (292, 21) (292,)


And save the data

In [7]:
import os
try:
  os.makedirs(name='outputs/datasets/most_important_features_data')
except Exception as e:
  print(e)

X_train.to_csv(f"outputs/datasets/training_data/X_train.csv",index=False)
X_val.to_csv(f"outputs/datasets/training_data/X_val.csv",index=False)
X_test.to_csv(f"outputs/datasets/training_data/X_test.csv",index=False)
y_train.to_csv(f"outputs/datasets/training_data/y_train.csv",index=False)
y_val.to_csv(f"outputs/datasets/training_data/y_val.csv",index=False)
y_test.to_csv(f"outputs/datasets/training_data/y_test.csv",index=False)


---

## Pipeline.

We can use the same pipeline as we used in the Modelling and Evaluation notebook, excepting that we can remove all features except 'GarageArea', 'GrLivArea', 'KitchenQual', 'OverallQual', 'TotalBsmtSF'.

In [10]:
from sklearn.pipeline import Pipeline
from feature_engine.selection import DropFeatures
from feature_engine.encoding import OrdinalEncoder
from feature_engine import transformation as vt
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import StandardScaler



pipeline = Pipeline([
    ('drop_features', DropFeatures(features_to_drop = ['1stFlrSF',
                                                        '2ndFlrSF',
                                                        'GarageYrBlt',
                                                        'YearBuilt',
                                                        'BsmtExposure',
                                                        'BsmtFinType1',
                                                        'GarageFinish',
                                                        'BedroomAbvGr',
                                                        'BsmtFinSF1',
                                                        'BsmtUnfSF',
                                                        'LotArea',
                                                        'LotFrontage',
                                                        'MasVnrArea',
                                                        'OpenPorchSF',
                                                        'OverallCond',
                                                        'YearRemodAdd']) ),

    ("OrdinalCategoricalEncoder",OrdinalEncoder(encoding_method='arbitrary', 
                                                  variables = ['KitchenQual'] ) ),

    ('pt', vt.PowerTransformer(variables = ['GarageArea', 'GrLivArea', 'OverallQual','TotalBsmtSF']) ),

    ('winsorizer_iqr', Winsorizer(capping_method='iqr', fold=1.5, tail='both')),
    
    ( "feat_scaling",StandardScaler() )
  ])

And use the pipeline to firt the train, validation and test sets.

In [11]:
X_train = pipeline.fit_transform(X_train)
X_val= pipeline.transform(X_val)
X_test = pipeline.transform(X_test)

---

# The Model

In [12]:
import os;
import tensorflow as tf;
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2';
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def create_tf_model(n_features):

  model = Sequential()
  model.add(Dense(units=n_features, activation='relu', input_shape=(n_features,)))

  model.add(Dense(units=512,activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(units=512,activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(units=512,activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(units=512,activation='relu'))
  model.add(Dropout(0.25))

  model.add(Dense(units=1))
  model.compile(loss='mse', optimizer='adam')
  
  return model

In [13]:
model = create_tf_model(n_features=X_train.shape[1])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 512)               3072      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)

In [15]:
model = create_tf_model(n_features=X_train.shape[1])

model.fit(x=X_train, 
          y=y_train, 
          epochs=100,
          validation_data=(X_val, y_val),
          verbose=1,
          callbacks=[early_stop]
          )

Epoch 1/100
30/30 [==============================] - 1s 31ms/step - loss: 39075577856.0000 - val_loss: 35612672000.0000
Epoch 2/100
30/30 [==============================] - 1s 21ms/step - loss: 23907575808.0000 - val_loss: 8323850240.0000
Epoch 3/100
30/30 [==============================] - 1s 20ms/step - loss: 5902743552.0000 - val_loss: 4221090560.0000
Epoch 4/100
30/30 [==============================] - 1s 21ms/step - loss: 3761725696.0000 - val_loss: 2940797952.0000
Epoch 5/100
30/30 [==============================] - 1s 20ms/step - loss: 2835893248.0000 - val_loss: 2305773056.0000
Epoch 6/100
30/30 [==============================] - 1s 18ms/step - loss: 2345599744.0000 - val_loss: 1872920064.0000
Epoch 7/100
30/30 [==============================] - 1s 17ms/step - loss: 2122716416.0000 - val_loss: 1666348416.0000
Epoch 8/100
30/30 [==============================] - 1s 22ms/step - loss: 1866425088.0000 - val_loss: 1611305728.0000
Epoch 9/100
30/30 [==============================] - 